# 네이버 영화리뷰 감정분석 with Hugging Face BERT
참고 소스 출처(링크) : https://github.com/deepseasw/bert-naver-movie-review

# 준비
라이브러리, 파라미터 세팅

In [1]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import os

In [2]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TEST_SIZE = 0.2
RANDOM_SEED = 42

In [3]:
# 디바이스 설정 확인
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 4 GPU(s) available.
We will use the GPU: GeForce RTX 2080 Ti


In [4]:
# GPU 할당 변경하기
GPU_NUM = 3 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
print ('Current cuda device ', torch.cuda.current_device()) # check

Current cuda device  3


# 데이터 로드

In [5]:
# 학습 데이터 로드
train = pd.read_csv(DATA_IN_PATH + 'ratings_train.txt', sep='\t')
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
# 검증(Original Test Dataset) 데이터 로드
dev = pd.read_csv(DATA_IN_PATH + 'ratings_test.txt', sep='\t')
dev.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [7]:
# 테스트(캐글) 데이터 로드
test = pd.read_csv(DATA_IN_PATH + 'ko_data.csv', encoding = 'cp949')
test.columns = ['id','document']  # 전처리 일괄 수행을 위해 컬럼명 변경(학습 데이터셋과 동일하게)
test.head()

,id,document
0,0,정말 많이 울었던 영화입니다.
1,1,시간 낭비예요.
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.


In [8]:
# 테이터 shape 확인
print(train.shape)
print(dev.shape)
print(test.shape)

(150000, 3)
(50000, 3)
(11187, 2)


In [9]:
# null 값 공백처리
train = train.fillna(' ')
dev = dev.fillna(' ')
test = test.fillna(' ')

In [10]:
# 데이터 타입 확인
print(train.dtypes)
print(dev.dtypes)
print(test.dtypes)

id           int64
document    object
label        int64
dtype: object
id           int64
document    object
label        int64
dtype: object
id           int64
document    object
dtype: object


# 데이터 전처리 - 학습 데이터셋

In [11]:
# 리뷰 문장 추출
sentences = train['document']
sentences[:10]

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
5        막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.
6                                원작의 긴장감을 제대로 살려내지못했다.
7    별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...
8                               액션이 없는데도 재미 있는 몇안되는 영화
9        왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?
Name: document, dtype: object

In [12]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]',
 '[CLS] 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [SEP]',
 '[CLS] 너무재밓었다그래서보는것을추천한다 [SEP]',
 '[CLS] 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [SEP]',
 '[CLS] 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP]',
 '[CLS] 막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움. [SEP]',
 '[CLS] 원작의 긴장감을 제대로 살려내지못했다. [SEP]',
 '[CLS] 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네 [SEP]',
 '[CLS] 액션이 없는데도 재미 있는 몇안되는 영화 [SEP]',
 '[CLS] 왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나? [SEP]']

![대체 텍스트](https://mino-park7.github.io/images/2019/02/bert-input-representation.png)

BERT의 입력은 위의 그림과 같은 형식. Classification을 뜻하는 [CLS] 심볼이 제일 앞에 삽입된다. 파인튜닝시 출력에서 이 위치의 값을 사용하여 분류를 수행한다. [SEP]은 Seperation을 가리키는데, 두 문장을 구분하는 역할을 한다. 이 예제에서는 문장이 하나이므로 [SEP]도 하나만 넣는다.
<br>
<br>

In [13]:
# 라벨 추출
labels = train['label'].values
labels

array([0, 1, 0, ..., 0, 1, 0])

In [14]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])


[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]
['[CLS]', '아', '더', '##빙', '.', '.', '진', '##짜', '짜', '##증', '##나', '##네', '##요', '목', '##소', '##리', '[SEP]']


<br>
BERT는 형태소분석으로 토큰을 분리하지 않는다. WordPiece라는 통계적인 방식을 사용한다. 한 단어내에서 자주 나오는 글자들을 붙여서 하나의 토큰으로 만든다. 이렇게 하면 언어에 상관없이 토큰을 생성할 수 있다는 장점이 있다. 또한 신조어 같이 사전에 없는 단어를 처리하기도 좋다. 

위의 결과에서 ## 기호는 앞 토큰과 이어진다는 표시이다. 토크나이저는 여러 언어의 데이터를 기반으로 만든 'bert-base-multilingual-cased'를 사용기에 한글도 처리가 가능하다.
<br>
<br>

In [15]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9519,   9074, 119005,    119,    119,   9708, 119235,
         9715, 119230,  16439,  77884,  48549,   9284,  22333,  12692,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

<br>
보통 딥러닝 모델에는 토큰 자체를 입력으로 넣을 수 없다. 임베딩 레이어에는 토큰을 숫자로 된 인덱스로 변환하여 사용한다. BERT의 토크나이저는 {단어토큰:인덱스}로 구성된 단어사전을 가지고 있다. 이를 참조하여 토큰을 인덱스로 바꿔준다.
<br>
<br>

In [16]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [17]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=RANDOM_SEED, 
                                                                                    test_size=TEST_SIZE)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=RANDOM_SEED, 
                                                       test_size=TEST_SIZE)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   8982,  10459,   9895,  39420,  11513,   9074, 118867,  18108,
         12508,  23811,    102,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [18]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 데이터 전처리 - 검증 데이터셋(Original Test Dataset)

In [19]:
# 리뷰 문장 추출
sentences = dev['document']
sentences[:10]

0                                                  굳 ㅋ
1                                 GDNTOPCLASSINTHECLUB
2               뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3                     지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4    3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
5                                   음악이 주가 된, 최고의 음악영화
6                                              진정한 쓰레기
7             마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다
8    갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...
9       이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..
Name: document, dtype: object

In [20]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 굳 ㅋ [SEP]',
 '[CLS] GDNTOPCLASSINTHECLUB [SEP]',
 '[CLS] 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아 [SEP]',
 '[CLS] 지루하지는 않은데 완전 막장임... 돈주고 보기에는.... [SEP]',
 '[CLS] 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠?? [SEP]',
 '[CLS] 음악이 주가 된, 최고의 음악영화 [SEP]',
 '[CLS] 진정한 쓰레기 [SEP]',
 '[CLS] 마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다 [SEP]',
 '[CLS] 갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다 [SEP]',
 '[CLS] 이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네.. [SEP]']

In [21]:
# 라벨 추출
labels = dev['label'].values
labels

array([1, 0, 0, ..., 0, 0, 0])

In [22]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 굳 ㅋ [SEP]
['[CLS]', '굳', '[UNK]', '[SEP]']


In [23]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([ 101, 8911,  100,  102,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [24]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [25]:
# 데이터를 파이토치의 텐서로 변환
dev_inputs = torch.tensor(input_ids)
dev_labels = torch.tensor(labels)
dev_masks = torch.tensor(attention_masks)

print(dev_inputs[0])
print(dev_labels[0])
print(dev_masks[0])

tensor([ 101, 8911,  100,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
tensor(1)
tensor([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [26]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
dev_data = TensorDataset(dev_inputs, dev_masks, dev_labels)
dev_sampler = RandomSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=batch_size)

# 모델 생성

In [27]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

![대체 텍스트](http://www.mccormickml.com/assets/BERT/padding_and_mask.png)

사전훈련된 BERT는 다양한 문제로 전이학습 가능. 여기서는 위의 그림과 같이 한 문장을 분류하는 방법을 사용. 영화리뷰 문장이 입력으로 들어가면, 긍정/부정으로 구분. 모델의 출력에서 [CLS] 위치인 첫 번째 토큰에 새로운 레이어를 붙여서 파인튜닝을 수행. Huggning Face는 BertForSequenceClassification() 함수를 제공하기 때문에 쉽게 구현 가능.
<br>
<br>

In [28]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# 모델 학습

In [29]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [30]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [31]:
# f1-score parameter
from sklearn.metrics import f1_score
f1_score_avg = []
trues = []
preds = []

In [32]:
# 재현을 위해 랜덤시드 고정
seed_val = RANDOM_SEED
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.5f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # F1-Score
        pred_flat = np.argmax(logits, axis=1).flatten()
        trues_flat = label_ids.flatten()
        trues.extend(trues_flat)
        preds.extend(pred_flat)
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
    print("  F1-Score macro: {0: 5f}".format(f1_score(y_true=trues, y_pred=preds, labels = [0,1], average='macro')))
    print("  F1-Score micro: {0: 5f}".format(f1_score(y_true=trues, y_pred=preds, labels = [0,1],average='micro')))
    print("  F1-Score weighted: {0: 5f}".format(f1_score(y_true=trues, y_pred=preds, labels = [0,1],average='weighted')))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  3,750.    Elapsed: 0:02:12.
  Batch 1,000  of  3,750.    Elapsed: 0:04:25.
  Batch 1,500  of  3,750.    Elapsed: 0:06:38.
  Batch 2,000  of  3,750.    Elapsed: 0:08:52.
  Batch 2,500  of  3,750.    Elapsed: 0:11:07.
  Batch 3,000  of  3,750.    Elapsed: 0:13:22.
  Batch 3,500  of  3,750.    Elapsed: 0:15:36.

  Average training loss: 0.39429
  Training epcoh took: 0:16:44

Running Validation...
  Accuracy: 0.85658
  f1 score macro:  0.856527
  f1 score micro:  0.856533
  f1 score weighted:  0.856516
  Validation took: 0:01:22

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  3,750.    Elapsed: 0:02:15.
  Batch 1,000  of  3,750.    Elapsed: 0:04:29.
  Batch 1,500  of  3,750.    Elapsed: 0:06:44.
  Batch 2,000  of  3,750.    Elapsed: 0:08:58.
  Batch 2,500  of  3,750.    Elapsed: 0:11:13.
  Batch 3,000  of  3,750.    Elapsed: 0:13:27.
  Batch 3,500  of  3,750.    Elapsed: 0:15:42.

  Average training loss: 0.28965


에폭마다 훈련셋과 검증셋을 반복하여 학습 수행. 

# 검증 데이터셋(Original Test Dataset) 평가

In [33]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(dev_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(dev_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # F1-Score
    pred_flat = np.argmax(logits, axis=1).flatten()
    trues_flat = label_ids.flatten()
    trues.extend(trues_flat)
    preds.extend(pred_flat)
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
print("F1-Score macro: {0: 5f}".format(f1_score(y_true=trues, y_pred= preds, labels = [0,1], average='macro')))
print("F1-Score micro: {0: 5f}".format(f1_score(y_true=trues, y_pred= preds, labels = [0,1],average='micro')))
print("F1-Score weighted: {0: 5f}".format(f1_score(y_true=trues, y_pred=preds, labels = [0,1],average='weighted')))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  1,563.    Elapsed: 0:00:09.
  Batch   200  of  1,563.    Elapsed: 0:00:18.
  Batch   300  of  1,563.    Elapsed: 0:00:27.
  Batch   400  of  1,563.    Elapsed: 0:00:36.
  Batch   500  of  1,563.    Elapsed: 0:00:45.
  Batch   600  of  1,563.    Elapsed: 0:00:54.
  Batch   700  of  1,563.    Elapsed: 0:01:02.
  Batch   800  of  1,563.    Elapsed: 0:01:11.
  Batch   900  of  1,563.    Elapsed: 0:01:20.
  Batch 1,000  of  1,563.    Elapsed: 0:01:29.
  Batch 1,100  of  1,563.    Elapsed: 0:01:37.
  Batch 1,200  of  1,563.    Elapsed: 0:01:46.
  Batch 1,300  of  1,563.    Elapsed: 0:01:54.
  Batch 1,400  of  1,563.    Elapsed: 0:02:03.
  Batch 1,500  of  1,563.    Elapsed: 0:02:12.

Accuracy: 0.87064
F1-Score macro:  0.866476
F1-Score micro:  0.866488
F1-Score weighted:  0.866469
Test took: 0:02:18


# (참고) 제출 파일 생성

In [34]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [35]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [36]:
test['document']

0                                         정말 많이 울었던 영화입니다.
1                                                 시간 낭비예요.
2                   포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3                     지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4                                이걸 영화로 만드는 거야?얼마나 가는지 보자.
                               ...                        
11182    이 영화를 커플에게 추천합니다. 영화관에 가다보면 평생 잊지 못할 추억이 하나 생길...
11183                                       심심__ 그냥 한효주 cf
11184    공감해서 눈물나는 영화. 안 보신분들이 전부 제가 울었다고 하면 의아해하실텐데 보면...
11185                                        오토바이 신은 최고네요.
11186                                     개병헌 쓰면 엉망이 된다ㅋㅋㅋ
Name: document, Length: 11187, dtype: object

In [37]:
import time
start = time.time()  # 시작 시간 저장

test_preds = []
for i, sentence in enumerate(test['document']):
    logits = test_sentences([sentence])
    test_preds.append(np.argmax(logits))
    print(str(i)+"번 인덱스 데이터 처리 완료 =>", logits, np.argmax(logits))
    
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

0번 인덱스 데이터 처리 완료 => [[-2.7851813  2.2655375]] 1
1번 인덱스 데이터 처리 완료 => [[ 3.4458556 -3.2661605]] 0
2번 인덱스 데이터 처리 완료 => [[ 0.97079474 -0.74195236]] 0
3번 인덱스 데이터 처리 완료 => [[-3.3735163  2.824754 ]] 1
4번 인덱스 데이터 처리 완료 => [[ 1.8795758 -1.5415624]] 0
5번 인덱스 데이터 처리 완료 => [[-3.4088192  2.7768838]] 1
6번 인덱스 데이터 처리 완료 => [[ 3.010643  -2.6370523]] 0
7번 인덱스 데이터 처리 완료 => [[-1.7689291  1.341521 ]] 1
8번 인덱스 데이터 처리 완료 => [[ 0.5218746 -0.345498 ]] 0
9번 인덱스 데이터 처리 완료 => [[-2.542486  2.048076]] 1
10번 인덱스 데이터 처리 완료 => [[-3.291205   2.6820006]] 1
11번 인덱스 데이터 처리 완료 => [[ 2.3906517 -2.1105762]] 0
12번 인덱스 데이터 처리 완료 => [[ 1.809383 -1.605401]] 0
13번 인덱스 데이터 처리 완료 => [[ 3.2533123 -3.0394945]] 0
14번 인덱스 데이터 처리 완료 => [[ 1.3396932 -1.0326346]] 0
15번 인덱스 데이터 처리 완료 => [[ 1.4157867 -1.2711447]] 0
16번 인덱스 데이터 처리 완료 => [[ 1.0132397 -0.889487 ]] 0
17번 인덱스 데이터 처리 완료 => [[ 1.5780137 -1.2725658]] 0
18번 인덱스 데이터 처리 완료 => [[-1.0620335   0.97516435]] 1
19번 인덱스 데이터 처리 완료 => [[-3.1612387  2.480978 ]] 1
20번 인덱스 데이터 처리 완료 => [[-3.3195

191번 인덱스 데이터 처리 완료 => [[-2.1873012  1.6776845]] 1
192번 인덱스 데이터 처리 완료 => [[-1.8180406  1.4202068]] 1
193번 인덱스 데이터 처리 완료 => [[ 3.3453007 -3.2152934]] 0
194번 인덱스 데이터 처리 완료 => [[ 1.444556  -1.2790034]] 0
195번 인덱스 데이터 처리 완료 => [[-2.3509026  1.9153352]] 1
196번 인덱스 데이터 처리 완료 => [[-2.396616  1.805746]] 1
197번 인덱스 데이터 처리 완료 => [[-0.42370418  0.15643483]] 1
198번 인덱스 데이터 처리 완료 => [[ 3.0223246 -2.624566 ]] 0
199번 인덱스 데이터 처리 완료 => [[-1.144508  0.907014]] 1
200번 인덱스 데이터 처리 완료 => [[ 3.4052725 -3.101259 ]] 0
201번 인덱스 데이터 처리 완료 => [[ 0.6959087  -0.47865143]] 0
202번 인덱스 데이터 처리 완료 => [[-2.4319644  1.9229224]] 1
203번 인덱스 데이터 처리 완료 => [[ 3.5428119 -3.2943559]] 0
204번 인덱스 데이터 처리 완료 => [[-2.3551996  1.9257021]] 1
205번 인덱스 데이터 처리 완료 => [[ 1.7129337 -1.3886719]] 0
206번 인덱스 데이터 처리 완료 => [[-2.9882994  2.3477683]] 1
207번 인덱스 데이터 처리 완료 => [[ 1.5456545 -1.2754691]] 0
208번 인덱스 데이터 처리 완료 => [[-1.9879946  1.5667961]] 1
209번 인덱스 데이터 처리 완료 => [[-3.117463   2.5032542]] 1
210번 인덱스 데이터 처리 완료 => [[ 3.0450735 -2.638601 ]] 0


373번 인덱스 데이터 처리 완료 => [[ 1.7388073 -1.5057003]] 0
374번 인덱스 데이터 처리 완료 => [[ 3.5371876 -3.221895 ]] 0
375번 인덱스 데이터 처리 완료 => [[ 1.1935278 -1.1116501]] 0
376번 인덱스 데이터 처리 완료 => [[-2.2935562  2.1155746]] 1
377번 인덱스 데이터 처리 완료 => [[-3.321886   2.6585064]] 1
378번 인덱스 데이터 처리 완료 => [[-1.4070505  1.1904674]] 1
379번 인덱스 데이터 처리 완료 => [[-1.4573439  1.2593076]] 1
380번 인덱스 데이터 처리 완료 => [[-3.179581   2.4848275]] 1
381번 인덱스 데이터 처리 완료 => [[ 3.1643014 -2.8820536]] 0
382번 인덱스 데이터 처리 완료 => [[-3.334089  2.653426]] 1
383번 인덱스 데이터 처리 완료 => [[ 2.6371045 -2.4520197]] 0
384번 인덱스 데이터 처리 완료 => [[-2.0882003  1.7254753]] 1
385번 인덱스 데이터 처리 완료 => [[-1.8725568  1.613023 ]] 1
386번 인덱스 데이터 처리 완료 => [[-0.65971476  0.6595889 ]] 1
387번 인덱스 데이터 처리 완료 => [[-1.9776598  1.5589476]] 1
388번 인덱스 데이터 처리 완료 => [[-0.49022427  0.33720586]] 1
389번 인덱스 데이터 처리 완료 => [[-1.6120956  1.2452794]] 1
390번 인덱스 데이터 처리 완료 => [[ 3.214025  -2.8574467]] 0
391번 인덱스 데이터 처리 완료 => [[-2.0477104  1.7233636]] 1
392번 인덱스 데이터 처리 완료 => [[-1.5399646  1.292967 ]] 

553번 인덱스 데이터 처리 완료 => [[-3.3160536  2.754304 ]] 1
554번 인덱스 데이터 처리 완료 => [[ 3.573282  -3.2507007]] 0
555번 인덱스 데이터 처리 완료 => [[ 2.6836379 -2.3472192]] 0
556번 인덱스 데이터 처리 완료 => [[-2.1792748  1.6812459]] 1
557번 인덱스 데이터 처리 완료 => [[-3.1005695  2.612879 ]] 1
558번 인덱스 데이터 처리 완료 => [[-2.9422657  2.395493 ]] 1
559번 인덱스 데이터 처리 완료 => [[ 3.512158  -3.2907937]] 0
560번 인덱스 데이터 처리 완료 => [[ 3.2424736 -2.9910598]] 0
561번 인덱스 데이터 처리 완료 => [[ 3.406662  -3.3625827]] 0
562번 인덱스 데이터 처리 완료 => [[-2.9852693  2.4171822]] 1
563번 인덱스 데이터 처리 완료 => [[-2.528559  2.062036]] 1
564번 인덱스 데이터 처리 완료 => [[-3.2096868  2.619879 ]] 1
565번 인덱스 데이터 처리 완료 => [[ 2.5954351 -2.4819894]] 0
566번 인덱스 데이터 처리 완료 => [[ 2.3295524 -2.0402937]] 0
567번 인덱스 데이터 처리 완료 => [[-3.0506663  2.415719 ]] 1
568번 인덱스 데이터 처리 완료 => [[ 2.932757 -2.64551 ]] 0
569번 인덱스 데이터 처리 완료 => [[ 3.335081 -3.336379]] 0
570번 인덱스 데이터 처리 완료 => [[ 1.9672745 -1.682714 ]] 0
571번 인덱스 데이터 처리 완료 => [[-3.405241  2.77156 ]] 1
572번 인덱스 데이터 처리 완료 => [[-2.99387    2.4361172]] 1
573번 인덱스

733번 인덱스 데이터 처리 완료 => [[-2.5885854  2.1074214]] 1
734번 인덱스 데이터 처리 완료 => [[-2.1010354  1.907992 ]] 1
735번 인덱스 데이터 처리 완료 => [[ 3.4792624 -3.236962 ]] 0
736번 인덱스 데이터 처리 완료 => [[ 1.7701107 -1.4773223]] 0
737번 인덱스 데이터 처리 완료 => [[-1.1653296  0.9625618]] 1
738번 인덱스 데이터 처리 완료 => [[-2.426083   1.9375578]] 1
739번 인덱스 데이터 처리 완료 => [[-1.6562433  1.2412887]] 1
740번 인덱스 데이터 처리 완료 => [[ 3.5444624 -3.312111 ]] 0
741번 인덱스 데이터 처리 완료 => [[-2.0413685  1.7507586]] 1
742번 인덱스 데이터 처리 완료 => [[-3.2228487  2.6383314]] 1
743번 인덱스 데이터 처리 완료 => [[-1.8663744  1.5733442]] 1
744번 인덱스 데이터 처리 완료 => [[-2.0219264  1.8291748]] 1
745번 인덱스 데이터 처리 완료 => [[-3.4568954  2.864035 ]] 1
746번 인덱스 데이터 처리 완료 => [[-2.9706278  2.4592175]] 1
747번 인덱스 데이터 처리 완료 => [[-2.0283935  1.7762028]] 1
748번 인덱스 데이터 처리 완료 => [[-3.2314935  2.852624 ]] 1
749번 인덱스 데이터 처리 완료 => [[ 1.1742376 -0.9978964]] 0
750번 인덱스 데이터 처리 완료 => [[ 0.55750203 -0.42654786]] 0
751번 인덱스 데이터 처리 완료 => [[-0.34096676  0.13451825]] 1
752번 인덱스 데이터 처리 완료 => [[-2.5824823  2.1575968]

...

10445번 인덱스 데이터 처리 완료 => [[ 0.8019728  -0.90663534]] 0
10446번 인덱스 데이터 처리 완료 => [[ 1.2326901  -0.90597993]] 0
10447번 인덱스 데이터 처리 완료 => [[-2.4580567  2.1223629]] 1
10448번 인덱스 데이터 처리 완료 => [[ 1.7445434 -1.5555875]] 0
10449번 인덱스 데이터 처리 완료 => [[ 1.0626757 -0.8071702]] 0
10450번 인덱스 데이터 처리 완료 => [[ 3.0758176 -2.7360165]] 0
10451번 인덱스 데이터 처리 완료 => [[-2.246244   2.1404254]] 1
10452번 인덱스 데이터 처리 완료 => [[-2.5598497  2.0842457]] 1
10453번 인덱스 데이터 처리 완료 => [[-2.8826876  2.1787667]] 1
10454번 인덱스 데이터 처리 완료 => [[ 0.8533827 -0.6308402]] 0
10455번 인덱스 데이터 처리 완료 => [[-2.5161076  1.9160893]] 1
10456번 인덱스 데이터 처리 완료 => [[ 3.2159111 -2.9187007]] 0
10457번 인덱스 데이터 처리 완료 => [[ 3.184405 -2.82149 ]] 0
10458번 인덱스 데이터 처리 완료 => [[ 0.86837137 -0.67826825]] 0
10459번 인덱스 데이터 처리 완료 => [[ 0.23917112 -0.15090056]] 0
10460번 인덱스 데이터 처리 완료 => [[-3.351784   2.9093366]] 1
10461번 인덱스 데이터 처리 완료 => [[-0.91542846  0.8993733 ]] 1
10462번 인덱스 데이터 처리 완료 => [[-3.4033709  2.886241 ]] 1
10463번 인덱스 데이터 처리 완료 => [[-3.01959    2.6014423]] 1
1046

10622번 인덱스 데이터 처리 완료 => [[-3.0661068  2.445933 ]] 1
10623번 인덱스 데이터 처리 완료 => [[-1.5213737  1.180034 ]] 1
10624번 인덱스 데이터 처리 완료 => [[-3.2700338  2.5124736]] 1
10625번 인덱스 데이터 처리 완료 => [[ 1.2890657 -1.0855074]] 0
10626번 인덱스 데이터 처리 완료 => [[-0.23192495  0.0692809 ]] 1
10627번 인덱스 데이터 처리 완료 => [[-1.7177387  1.3904418]] 1
10628번 인덱스 데이터 처리 완료 => [[ 1.9146632 -1.5790304]] 0
10629번 인덱스 데이터 처리 완료 => [[-3.1031473  2.5898209]] 1
10630번 인덱스 데이터 처리 완료 => [[ 0.5743243  -0.45155144]] 0
10631번 인덱스 데이터 처리 완료 => [[-2.5884657  2.0882232]] 1
10632번 인덱스 데이터 처리 완료 => [[ 1.6180896 -1.2615364]] 0
10633번 인덱스 데이터 처리 완료 => [[-2.0420425  1.8065519]] 1
10634번 인덱스 데이터 처리 완료 => [[ 3.18264   -2.9615283]] 0
10635번 인덱스 데이터 처리 완료 => [[ 1.7367388 -1.5407962]] 0
10636번 인덱스 데이터 처리 완료 => [[ 2.7362225 -2.4855933]] 0
10637번 인덱스 데이터 처리 완료 => [[-1.8827622  1.5025271]] 1
10638번 인덱스 데이터 처리 완료 => [[-2.4864666  1.8823816]] 1
10639번 인덱스 데이터 처리 완료 => [[ 1.3074952 -1.0456   ]] 0
10640번 인덱스 데이터 처리 완료 => [[ 0.41258228 -0.37490812]] 0
10641번

10797번 인덱스 데이터 처리 완료 => [[-3.2715232  2.75693  ]] 1
10798번 인덱스 데이터 처리 완료 => [[-3.3427505  2.82958  ]] 1
10799번 인덱스 데이터 처리 완료 => [[ 3.4731653 -3.2915251]] 0
10800번 인덱스 데이터 처리 완료 => [[ 1.7045498 -1.5421455]] 0
10801번 인덱스 데이터 처리 완료 => [[-2.557476   1.9745747]] 1
10802번 인덱스 데이터 처리 완료 => [[-0.24529395  0.4302666 ]] 1
10803번 인덱스 데이터 처리 완료 => [[ 3.1484919 -2.8025966]] 0
10804번 인덱스 데이터 처리 완료 => [[ 3.408932 -3.11021 ]] 0
10805번 인덱스 데이터 처리 완료 => [[-0.8963427  0.6841169]] 1
10806번 인덱스 데이터 처리 완료 => [[-2.959716   2.3828282]] 1
10807번 인덱스 데이터 처리 완료 => [[ 3.4441032 -3.3814707]] 0
10808번 인덱스 데이터 처리 완료 => [[ 3.3043563 -2.9189446]] 0
10809번 인덱스 데이터 처리 완료 => [[ 1.6861132 -1.3414186]] 0
10810번 인덱스 데이터 처리 완료 => [[-2.648888   2.0932922]] 1
10811번 인덱스 데이터 처리 완료 => [[ 1.906505  -1.6307015]] 0
10812번 인덱스 데이터 처리 완료 => [[ 3.102892  -2.7312953]] 0
10813번 인덱스 데이터 처리 완료 => [[-2.8805938  2.4672112]] 1
10814번 인덱스 데이터 처리 완료 => [[-2.661312   2.1310906]] 1
10815번 인덱스 데이터 처리 완료 => [[-3.2355525  2.4559872]] 1
10816번 인덱스 데

10972번 인덱스 데이터 처리 완료 => [[-1.554592   1.3364409]] 1
10973번 인덱스 데이터 처리 완료 => [[-3.1925712  2.7632697]] 1
10974번 인덱스 데이터 처리 완료 => [[-2.999682   2.3583026]] 1
10975번 인덱스 데이터 처리 완료 => [[-2.5510368  2.1634243]] 1
10976번 인덱스 데이터 처리 완료 => [[-2.1190658  1.9085243]] 1
10977번 인덱스 데이터 처리 완료 => [[-1.6391414  1.4090146]] 1
10978번 인덱스 데이터 처리 완료 => [[-1.7982032  1.6604639]] 1
10979번 인덱스 데이터 처리 완료 => [[ 2.946013  -2.6776655]] 0
10980번 인덱스 데이터 처리 완료 => [[ 3.4983358 -3.244701 ]] 0
10981번 인덱스 데이터 처리 완료 => [[-2.9998674  2.4249694]] 1
10982번 인덱스 데이터 처리 완료 => [[ 0.8610346  -0.69570166]] 0
10983번 인덱스 데이터 처리 완료 => [[-2.6142051  2.3847482]] 1
10984번 인덱스 데이터 처리 완료 => [[-3.0029855  2.3656392]] 1
10985번 인덱스 데이터 처리 완료 => [[-2.4301147  1.8828309]] 1
10986번 인덱스 데이터 처리 완료 => [[ 1.5289398 -1.3008721]] 0
10987번 인덱스 데이터 처리 완료 => [[ 3.1107635 -2.780715 ]] 0
10988번 인덱스 데이터 처리 완료 => [[-3.3164067  2.712167 ]] 1
10989번 인덱스 데이터 처리 완료 => [[-1.6863884  1.3696928]] 1
10990번 인덱스 데이터 처리 완료 => [[ 1.5048307 -1.1493938]] 0
10991번 인덱스

11151번 인덱스 데이터 처리 완료 => [[ 1.3509152 -1.1319535]] 0
11152번 인덱스 데이터 처리 완료 => [[ 3.3480203 -3.0359626]] 0
11153번 인덱스 데이터 처리 완료 => [[ 2.6247249 -2.2422268]] 0
11154번 인덱스 데이터 처리 완료 => [[-3.3652039  2.8466215]] 1
11155번 인덱스 데이터 처리 완료 => [[ 2.9958484 -2.8300362]] 0
11156번 인덱스 데이터 처리 완료 => [[-2.9039397  2.2234066]] 1
11157번 인덱스 데이터 처리 완료 => [[ 0.3217283  -0.22140053]] 0
11158번 인덱스 데이터 처리 완료 => [[ 3.2136214 -2.8870602]] 0
11159번 인덱스 데이터 처리 완료 => [[ 3.245548  -3.1412668]] 0
11160번 인덱스 데이터 처리 완료 => [[-0.50242287  0.39727423]] 1
11161번 인덱스 데이터 처리 완료 => [[-3.0525303  2.576324 ]] 1
11162번 인덱스 데이터 처리 완료 => [[-2.6348119  2.187946 ]] 1
11163번 인덱스 데이터 처리 완료 => [[-3.3028708  2.8670087]] 1
11164번 인덱스 데이터 처리 완료 => [[ 3.4694521 -3.169311 ]] 0
11165번 인덱스 데이터 처리 완료 => [[ 2.3273904 -2.019638 ]] 0
11166번 인덱스 데이터 처리 완료 => [[-2.8296595  2.3372235]] 1
11167번 인덱스 데이터 처리 완료 => [[-2.5433888  2.0883043]] 1
11168번 인덱스 데이터 처리 완료 => [[ 3.1541872 -2.7453547]] 0
11169번 인덱스 데이터 처리 완료 => [[ 3.4513912 -3.2890108]] 0
11170번 인

In [38]:
len(test_preds)

11187

In [39]:
# 테스트 데이터의 리뷰 부분을 리스트 처리
test_id = list(test['id'])

# 판다스 데이터프레임 통해 데이터 구성하여 output에 투입
output = pd.DataFrame( data={"Id": test_id, "Predicted": test_preds} )
output.head()

,Id,Predicted
0,0,1
1,1,0
2,2,0
3,3,1
4,4,0


In [40]:
# 해당 경로가 없으면 생성
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

# csv파일로 만든다
output.to_csv(DATA_OUT_PATH + "NSMC_BERT.csv", index = False)  # 앙상블 조합 및 캐글 제출 용도

### 캐글 제출 결과
**[2020.12.09]**<br>
0.83550<br>

**[2020.12.20]**<br>
0.83586

# (참고) 문장 테스트

In [41]:
logits = test_sentences(['연기는 별로지만 재미 하나는 끝내줌!'])

print(logits)
print(np.argmax(logits))

[[-0.9339185  0.7408574]]
1


In [42]:
logits = test_sentences(['주연배우가 아깝다. 총체적 난국...'])

print(logits)
print(np.argmax(logits))

[[ 3.2740705 -3.2614126]]
0


학습한 모델을 가지고 실제 문장을 넣었을 때 출력 로짓은 소프트맥스가 적용되지 않은 상태로, argmax로 더 높은 값의 위치를 라벨로 설정한 결과의 0은 부정, 1은 긍정이다.